In [515]:
import sqlite3
import pandas as pd

In [516]:
con = sqlite3.connect(':memory:')

In [517]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,NaN,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,NaN,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [518]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [519]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,NaN,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,NaN,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [520]:
tasks = 0

# Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [521]:

#пустые строки
pd.read_sql(
    '''
    SELECT count(*)-count(CRIM) as CRIM , count(*)-count(ZN) as ZN,count(*)-count(INDUS) as INDUS, 
    count(*)-count(CHAS) as CHAS, count(*)-count(NOX) as NOX FROM boston
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,0,1,0,2,0


In [522]:
#нулевые строки
tasks+=1
pd.read_sql(
    '''
    WITH temporarytable1 AS (SELECT count(CRIM) as CRIM FROM boston
    where CRIM =0),
    temporarytable2 AS (SELECT count(ZN) as ZN FROM boston
    where ZN =0 ),
    temporarytable3 AS (SELECT count(INDUS) as INDUS FROM boston
    where INDUS =0),
    temporarytable4 AS (SELECT count(CHAS) as CHAS FROM boston
    where CHAS =0 ),
    temporarytable5 AS (SELECT count(NOX) as NOX FROM boston
    where NOX =0)
    SELECT *  from temporarytable1 
    join temporarytable2
    join temporarytable3
    join temporarytable4
    join temporarytable5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,0,372,0,469,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [523]:
tasks+=1
#
pd.read_sql(
    '''
    SELECT count(DISTINCT CRIM) as CRIM , count(DISTINCT ZN) as ZN,count(DISTINCT INDUS) as INDUS, 
    count(DISTINCT CHAS) as CHAS, count(DISTINCT NOX) as NOX FROM boston
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,25,76,2,81


3.Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.

In [524]:

tasks+=1

#
pd.read_sql(
    '''
    
    SELECT DISTINCT  (SELECT name FROM pragma_table_info('boston') where name = 'ZN') as colomn_name FROM boston
    where (SELECT ZN FROM boston
    ORDER BY CRIM
    LIMIT 1
    OFFSET ROUND((SELECT COUNT(*)
    FROM boston) * 0,5)) =(SELECT min(ZN) FROM boston)
    
    UNION 
    
    SELECT DISTINCT (SELECT name FROM pragma_table_info('boston') where name = 'CRIM') as colomn_name  FROM boston
    where (SELECT CRIM FROM boston
    ORDER BY CRIM
    LIMIT 1
    OFFSET (SELECT COUNT(*)
    FROM boston) / 2) =(SELECT min(CRIM) FROM boston)
    UNION 
    
    SELECT DISTINCT (SELECT name FROM pragma_table_info('boston') where name = 'INDUS') as colomn_name  FROM boston
    where (SELECT CRIM FROM boston
    ORDER BY INDUS
    LIMIT 1
    OFFSET (SELECT COUNT(*)
    FROM boston) / 2) =(SELECT min(INDUS) FROM boston)
    UNION 
    
    SELECT DISTINCT (SELECT name FROM pragma_table_info('boston') where name = 'CHAS') as colomn_name  FROM boston
    where(SELECT CHAS FROM boston
    ORDER BY CHAS
    LIMIT 1
    OFFSET (SELECT COUNT(*)
    FROM boston) / 2) =(SELECT min(CHAS) FROM boston)
    UNION 
    
    SELECT DISTINCT (SELECT name FROM pragma_table_info('boston') where name = 'NOX') as colomn_name  FROM boston
    where (SELECT NOX FROM boston
    ORDER BY NOX
    LIMIT 1
    OFFSET (SELECT COUNT(*)
    FROM boston) / 2) =(SELECT min(NOX) FROM boston)
    ''',
    con,
)

,colomn_name
0,CHAS


4. Выведите разницу между средним количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами.
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница). Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [525]:

#сравнение кол-ва комнат 25 самых дешевых домов и самого дорогого
pd.read_sql(
    '''
    WITH tt1 AS (
    SELECT avg(RM) as exp FROM boston
    where MEDV = (SELECT max(MEDV) FROM boston)
    ), 
    tt2 AS (SELECT AVG(RM) as cheap FROM  (SELECT RM FROM boston 
    order by MEDV
    limit 50)
    )
    
    SELECT exp, cheap, (exp-cheap) as difference FROM tt1
    full outer join tt2
    limit 1
    
    
    ''',
    con,
)

,exp,cheap,difference
0,7.484,5.75324,1.73076


In [526]:


#Напишите влияет ли кол-во комнат на стоимость и как сильно.
pd.read_sql(
    '''
    WITH tt1 AS (
    SELECT avg(RM)  as rooms_expensive, AVG(MEDV) as cost_expensive FROM boston
    where MEDV = (SELECT max(MEDV) FROM boston)
    ), 
    tt2 AS (SELECT AVG(RM) as rooms_cheap, AVG(MEDV) as cost_cheap FROM  (SELECT RM, MEDV FROM boston 
    order by MEDV
    limit 25)
    )
    
    SELECT *, (rooms_expensive-rooms_cheap) as 'Разница', 
    rooms_expensive/rooms_cheap as 'Коэф. соотн. колич. комнат, люкс/дешевое',
    cost_expensive/cost_cheap as 'Коэф. соотношения цен, люкс/дешевое'
    FROM tt1
    full outer join tt2
    limit 1
    
    
    ''',
    con,
)



,rooms_expensive,cost_expensive,rooms_cheap,cost_cheap,Разница,"Коэф. соотн. колич. комнат, люкс/дешевое","Коэф. соотношения цен, люкс/дешевое"
0,7.484,50.0,5.74784,7.848,1.73616,1.302054,6.37105


Вывод: Коэффициент соотношения цен дорогой недвижимости к дешевой значительно выше, 
чем соответствующий коэффициент соотношения комнат,
Это значит, что количество комнат на стоимость хоть и влияет, но есть другие, более весомые факторы
 

In [527]:

tasks+=1
#сравнение кол-ва комнат 50 самых дешевых домов и самого дорогого
pd.read_sql(
    '''
    WITH tt1 AS (
    SELECT avg(RM) as expensive FROM boston
    where MEDV = (SELECT max(MEDV) FROM boston)
    ), 
    tt2 AS (SELECT AVG(RM) as cheap FROM  (SELECT RM FROM boston 
    order by MEDV
    limit 50)
    )
    
    SELECT expensive, cheap, (expensive-cheap) as difference FROM tt1
    full outer join tt2
    limit 1
    
    
    ''',
    con,
)

,expensive,cheap,difference
0,7.484,5.75324,1.73076


In [528]:

#сравнение кол-ва комнат 100 самых дешевых домов и самого дорогого
pd.read_sql(
    '''
    WITH tt1 AS (
    SELECT avg(RM) as expensive FROM boston
    where MEDV = (SELECT max(MEDV) FROM boston)
    ), 
    tt2 AS (SELECT AVG(RM) as cheap FROM  (SELECT RM FROM boston 
    order by MEDV
    limit 100)
    )
    
    SELECT expensive, cheap, (expensive-cheap) as difference FROM tt1
    full outer join tt2
    limit 1
    
    
    ''',
    con,
)

,expensive,cheap,difference
0,7.484,5.88712,1.59688


In [529]:

#сравнение кол-ва комнат 200 самых дешевых домов и самого дорогого
pd.read_sql(
    '''
    WITH tt1 AS (
    SELECT avg(RM) as expensive FROM boston
    where MEDV = (SELECT max(MEDV) FROM boston)
    ), 
    tt2 AS (SELECT AVG(RM) as cheap FROM  (SELECT RM FROM boston 
    order by MEDV
    limit 200)
    )
    
    SELECT expensive, cheap, (expensive-cheap) as difference FROM tt1
    full outer join tt2
    limit 1
    
    
    ''',
    con,
)

,expensive,cheap,difference
0,7.484,5.911705,1.572295


In [530]:

#сравнение кол-ва комнат 300 самых дешевых домов и самого дорогого
pd.read_sql(
    '''
    WITH tt1 AS (
    SELECT avg(RM) as expensive FROM boston
    where MEDV = (SELECT max(MEDV) FROM boston)
    ), 
    tt2 AS (SELECT AVG(RM) as cheap FROM  (SELECT RM FROM boston 
    order by MEDV
    limit 300)
    )
    
    SELECT expensive, cheap, (expensive-cheap) as difference FROM tt1
    full outer join tt2
    limit 1
    
    
    ''',
    con,
)

,expensive,cheap,difference
0,7.484,5.972227,1.511773


5.Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [531]:
tasks+=1
#
pd.read_sql(
    '''
    SELECT LSTAT , MEDV, RANK() OVER (ORDER BY LSTAT ) as RANG FROM boston
    ORDER BY LSTAT
    ''',
    con,
)


,LSTAT,MEDV,RANG
0,1.73,50.0,1
1,1.92,50.0,2
2,1.98,34.9,3
3,2.47,41.7,4
4,2.87,36.4,5
...,...,...,...
501,34.37,17.9,502
502,34.41,14.4,503
503,34.77,13.8,504
504,36.98,7.0,505


Вывод: чем выше стоимость недвижимости, тем меньше населения с низким социальным статусом там селится.

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [532]:
tasks+=1
#
pd.read_sql(
    '''
    SELECT AVG(MEDV) as "Средняя стоимость домов",
    CASE
    WHEN CHAS= 0 THEN 'Не граничит'
    WHEN CHAS= 1 THEN 'Граничит'
    ELSE 'Не определено'
    END "Граничит ли с рекой"

    FROM boston
    GROUP BY CHAS
    ''',
    con,
)


,Средняя стоимость домов,Граничит ли с рекой
0,32.45000,Не определено
1,22.04968,Не граничит
2,28.44000,Граничит


7.Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [533]:

#
pd.read_sql(
    '''
    SELECT AVG(MEDV) as "Средняя стоимость домов",
    AVG(CRIM),
    AVG(ZN),
    AVG(INDUS) "Доля промышленной застройки",
    AVG(NOX) as "концентрации оксидов азота",
    CASE
    WHEN CHAS= 0 THEN 'Не граничит'
    WHEN CHAS= 1 THEN 'Граничит'
    ELSE 'Не определено'
    END "Граничит ли с рекой"

    FROM boston
    GROUP BY CHAS
    ''',
    con,
)


,Средняя стоимость домов,AVG(CRIM),AVG(ZN),Доля промышленной застройки,концентрации оксидов азота,Граничит ли с рекой
0,32.45000,0.049450,0.000000,2.180000,0.458000,Не определено
1,22.04968,3.738882,11.670940,11.056887,0.552217,Не граничит
2,28.44000,1.851670,7.714286,12.719143,0.593426,Граничит


Вывод: для показателей, у который среднее значение значительно отличается, когда объект граничит с рекой,очевидно, характерно то, что возможно наличие корреляции этих показателей. Например, средняя стоимость выше,если недвижимость рядом с рекой.

In [534]:
tasks+=1
#
pd.read_sql(
    '''
    
    WITH tt AS (SELECT AVG(MEDV) as AVG_MEDV,
    AVG(CRIM) as AVG_CRIM,
    AVG(ZN) as AVG_ZN,
    AVG(INDUS) as AVG_INDUS,
    AVG(NOX) as AVG_NOX,
    CASE
    WHEN CHAS= 0 THEN 'Не граничит'
    WHEN CHAS= 1 THEN 'Граничит'
    ELSE 'Не определено'
    END river

    FROM boston
    GROUP BY CHAS
    ORDER BY river
    LIMIT 2)
    
    SELECT * FROM 
    
    (SELECT 
    CASE
    WHEN river = 'Граничит' AND AVG_MEDV > AVG(AVG_MEDV) THEN 'MEDV'
    ELSE ""
    END colomn_name
    FROM tt
    
    UNION
    
    SELECT  
    CASE
    WHEN river = 'Граничит' AND AVG_CRIM > AVG(AVG_CRIM) THEN 'CRIM'
    ELSE ""
    END colomn_name
    FROM tt  
    
    UNION
    
    SELECT  
    CASE
    WHEN river = 'Граничит' AND AVG_ZN > AVG(AVG_ZN) THEN 'ZN'
    ELSE ""
    END colomn_name
    FROM tt 
    
    UNION 
    
    SELECT  
    CASE
    WHEN river = 'Граничит' AND AVG_INDUS > AVG(AVG_INDUS) THEN 'INDUS'
    ELSE ""
    END colomn_name
    FROM tt 
        
    UNION 
    
    SELECT  
    CASE
    WHEN river = 'Граничит' AND AVG_NOX > AVG(AVG_NOX) THEN 'NOX'
    ELSE ""
    END colomn_name
    FROM tt) 
    
    where colomn_name <>""
    
    ''',
    con,
)


,colomn_name
0,INDUS
1,MEDV
2,NOX


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [535]:
tasks+=1
#
pd.read_sql(
    '''
    
    
    SELECT INDUS as "Промышленная застройка", NOX as "Концентрация оксидов азота",  
    CASE

    
    
    WHEN NOX = (SELECT MAX(NOX) FROM boston) THEN '100'  
    
    WHEN NOX >= (SELECT NOX FROM boston
    ORDER BY NOX
    LIMIT 1
    OFFSET ROUND((SELECT COUNT(*)
    FROM boston) * 0.9)-1) THEN '90'
    
    WHEN NOX >= (SELECT NOX FROM boston
    ORDER BY NOX
    LIMIT 1
    OFFSET ROUND((SELECT COUNT(*)
    FROM boston) * 0.8)-1) THEN '80'
    
    WHEN NOX>= (SELECT NOX FROM boston
    ORDER BY NOX
    LIMIT 1
    OFFSET ROUND((SELECT COUNT(*)
    FROM boston) * 0.7)-1) THEN '70'
    
    WHEN NOX>= (SELECT NOX FROM boston
    ORDER BY NOX
    LIMIT 1
    OFFSET ROUND((SELECT COUNT(*)
    FROM boston) * 0.6)-1) THEN '60'    

    WHEN NOX>= (SELECT NOX FROM boston
    ORDER BY NOX
    LIMIT 1
    OFFSET ROUND((SELECT COUNT(*)
    FROM boston) * 0.5)-1) THEN '50'
    
    WHEN NOX >= (SELECT NOX FROM boston
    ORDER BY NOX
    LIMIT 1
    OFFSET ROUND((SELECT COUNT(*)
    FROM boston) *0.4)-1) THEN '40'
    
    WHEN NOX>= (SELECT NOX FROM boston
    ORDER BY NOX
    LIMIT 1
    OFFSET ROUND((SELECT COUNT(*)
    FROM boston) * 0.3)-1) THEN '30'  
    
    WHEN NOX>= (SELECT NOX FROM boston
    ORDER BY NOX
    LIMIT 1
    OFFSET ROUND((SELECT COUNT(*)
    FROM boston) * 0.2)-1) THEN '20'    

    WHEN NOX>= (SELECT NOX FROM boston
    ORDER BY NOX
    LIMIT 1
    OFFSET ROUND((SELECT COUNT(*)
    FROM boston) * 0.1)-1) THEN '10'

    WHEN NOX = (SELECT MIN(NOX) FROM boston) THEN '0'


    ELSE '0-10'
    END "Процентиль концентрации оксидов азота"
    
    FROM boston
    ORDER BY NOX
    
    ''',
    con,
)


,Промышленная застройка,Концентрация оксидов азота,Процентиль концентрации оксидов азота
0,1.76,0.385,0
1,2.25,0.389,0-10
2,3.64,0.392,0-10
3,3.64,0.392,0-10
4,3.75,0.394,0-10
...,...,...,...
501,19.58,0.871,100
502,19.58,0.871,100
503,19.58,0.871,100
504,19.58,0.871,100


Вывод: Концентрация оксидов азота выше, когда больше объектов промышленной застройки.
 Перцентили более наглядно показывают, сколько значений больше.

In [536]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
